# Extract and save landscape characteristics associated with SNOTEL stations & CSO obs

import json

import requests
import numpy as np

import pandas as pd
import geopandas as gpd
from shapely import geometry as sgeom
import rasterio as rio
import xarray as xr
from affine import Affine
from scipy import ndimage

In [19]:
from paths import *
import richdem as rd
from osgeo import gdal
import rasterio as rio
import numpy as np
from scipy import ndimage
import requests
import geopandas as gpd
from Depth2SWE import swe_calc
import pandas as pd

In [10]:
# function to extract and save landscape characteristics 
# associated with SNOTEL stations
def extract_meta(gdf,mod_proj,dem_path,lc_path):
    '''
    gdf = geodataframe of SNOTEL in the domain
    
    proj = projection of the modeling domain 
    
    dem_path = path to digital elevation model of domain
    
    lc_path = path to nlcd landcover data of domain
    '''
    
    new=gdf.to_crs(mod_proj)
    
    #add x y values to CSO gdf
    gdf['x']=new.geometry.x
    gdf['y']=new.geometry.y
    
    #build list of coordinates from point geodataframe
    xy = list(map(list, zip(new.geometry.x,new.geometry.y)))
    
    #ELEVATION
    # DEM data
    src = rio.open(dem_path)
    #with rio.open(dtm) as src:
    elevation = src.read(1)

    #sample dem
    with rio.open(dem_path) as src:
        gdf['dem_elev'] = [sample[0] for sample in src.sample(xy)]
    #-----------------------------------------------------------    
    #SLOPE
    #read in data
    ds = gdal.Open(dem_path);
    data = np.array(ds.GetRasterBand(1).ReadAsArray());
    rda = rd.rdarray(data, no_data=-9999);
    slope = rd.TerrainAttribute(rda, attrib='slope_riserun');
    #get indicies
    with rio.open(dem_path) as src:
        rows, cols = rio.transform.rowcol(src.transform, new.geometry.centroid.x, new.geometry.centroid.y)
    #sample slope array
    gdf['slope'] = slope[rows,cols]
    
    #-----------------------------------------------------------        
    #ASPECT
    aspect = rd.TerrainAttribute(rda, attrib='aspect');
    
    #4-aspect key
    #0=N, 2=E, 4=S, 6=W, 8=flat
    DIR=aspect
    DIR[(DIR>=0) & (DIR<=45)]=0
    DIR[(DIR>45) & (DIR<=135)]=2
    DIR[(DIR>135) & (DIR<=225)]=4
    DIR[(DIR>225) & (DIR<=315)]=6
    DIR[(DIR>315) & (DIR<=360)]=0
    DIR[slope < 0.5]=8
    DIR.astype(int)
    
#     #8-aspect key
#     #0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW, 8=flat
#     DIR=aspect
#     DIR[(DIR>=0) & (DIR<=22.5)]=0
#     DIR[(DIR>22.5) & (DIR<=67.5)]=1
#     DIR[(DIR>67.5) & (DIR<=112.5)]=2
#     DIR[(DIR>112.5) & (DIR<=157.5)]=3
#     DIR[(DIR>157.5) & (DIR<=202.5)]=4
#     DIR[(DIR>202.5) & (DIR<=247.5)]=5
#     DIR[(DIR>247.5) & (DIR<=292.5)]=6
#     DIR[(DIR>292.5) & (DIR<=337.5)]=7
#     DIR[(DIR>337.5) & (DIR<=360)]=0
#     DIR[slope < 0.5]=8
#     DIR.astype(int)

    #sample aspect array
    gdf['aspect'] = DIR[rows,cols]
    
    #-----------------------------------------------------------    
    #LANDCOVER
    # LC data
    src = rio.open(lc_path)
    lc = src.read(1)

    # reassign lc from NLCD to SM classes
    DIR=DIR=np.empty([np.shape(lc)[0],np.shape(lc)[1]])
    DIR[lc == 11 ]=24
    DIR[lc == 12 ]=20
    DIR[lc == 21 ]=21
    DIR[lc == 22 ]=21
    DIR[lc == 23 ]=21
    DIR[lc == 24 ]=21
    DIR[lc == 31 ]=18
    DIR[lc == 41 ]=2
    DIR[lc == 42 ]=1
    DIR[lc == 43 ]=6
    DIR[lc == 51 ]=6
    DIR[lc == 52 ]=6
    DIR[lc == 71 ]=12
    DIR[lc == 72 ]=12
    DIR[lc == 73 ]=12
    DIR[lc == 74 ]=12
    DIR[lc == 81 ]=23
    DIR[lc == 82 ]=22
    DIR[lc == 90 ]=9
    DIR[lc == 95 ]=9
    DIR.astype(int)

    #sample lc
    gdf['lc'] = DIR[rows,cols]    

    #-----------------------------------------------------------    
    #TERRAIN COMPLEXITY
    # calculate terrain complexity 
    tc = ndimage.generic_filter(elevation, np.std, size=3)
    # sample tc
    gdf['tc'] = tc[rows,cols]
    
    return gdf

# SNOTEL

In [11]:
snotel_gdf = extract_meta(gdf,mod_proj,dem_path,lc_path)
# save somewhere

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)


,code,longitude,latitude,name,elevation_m,easting,northing,geometry,x,y,dem_elev,slope,aspect,lc,tc
0,314_WY_SNTL,-110.445442,43.940189,Base Camp,2151.887939453125,544505.845453,4.865379e+06,POINT (-110.44544 43.94019),544505.845453,4.865379e+06,2145,14.826075,2.0,1.0,12
1,347_MT_SNTL,-111.128029,44.508320,Black Bear,2490.216064453125,489823.440274,4.928341e+06,POINT (-111.12803 44.50832),489823.440274,4.928341e+06,2493,1.510381,4.0,1.0,2
2,350_WY_SNTL,-109.793327,44.376671,Blackwater,2980.944091796875,596129.923439,4.914418e+06,POINT (-109.79333 44.37667),596129.923439,4.914418e+06,3006,18.073633,2.0,1.0,15
3,353_WY_SNTL,-110.609734,42.964001,Blind Bull Sum,2636.52001953125,531828.554679,4.756891e+06,POINT (-110.60973 42.96400),531828.554679,4.756891e+06,2716,0.637377,0.0,1.0,3
4,419_WY_SNTL,-110.814819,42.645901,Cottonwood Creek,2337.81591796875,515180.157295,4.721511e+06,POINT (-110.81482 42.64590),515180.157295,4.721511e+06,2351,14.629807,6.0,1.0,18


In [22]:
def get_cso(st, ed, Bbox):
    #Issue CSO API observations request and load the results into a GeoDataFrame
    params = {
      "bbox": f"{Bbox['lonmin']},{Bbox['latmax']},{Bbox['lonmax']},{Bbox['latmin']}",
      "start_date": st,
      "end_date": ed,
      "format": "geojson",
      "limit": 5000,
    }

    csodata_resp = requests.get("https://api.communitysnowobs.org/observations", params=params)
    csodatajson = csodata_resp.json()
    #turn into geodataframe
    gdf = gpd.GeoDataFrame.from_features(csodatajson, crs=stn_proj)
    
    mask = (gdf['timestamp'] >= st) & (gdf['timestamp'] <= ed)
    gdf = gdf.loc[mask]
    gdf=gdf.reset_index(drop=True)
    print('Total number of CSO in daimain = ',len(gdf))
    
    ingdf = extract_meta(gdf,mod_proj,dem_path,lc_path)
    
    #need to format data for Hs_to_SWE conversion
    ingdf['dt'] = pd.to_datetime(ingdf['timestamp'], format='%Y-%m-%dT%H:%M:%S')
    ingdf['dt'] = pd.to_datetime(ingdf['dt']).dt.date
    ingdf['Y'] = pd.DatetimeIndex(ingdf['dt']).year
    ingdf['M'] = pd.DatetimeIndex(ingdf['dt']).month
    ingdf['D'] = pd.DatetimeIndex(ingdf['dt']).day
    ingdf["LON"] = ingdf.geometry.x
    ingdf["LAT"] = ingdf.geometry.y
    ingdf=ingdf.drop(columns=['dt'])
    
    #convert snow depth to mm to input into density function
    ingdf['H'] = ingdf.depth*10
    ingdf.head()
    
    #Hs to SWE
    SWE,DOY = swe_calc(CSO_gdf.Y.values,CSO_gdf.M.values,CSO_gdf.D.values,CSO_gdf.H.values,CSO_gdf.LAT.values,CSO_gdf.LON.values)
    return ingdf

st = '2018-10-01'
ed = '2019-09-30'

csogdf = get_cso(st, ed, Bbox)

# save somewhere

Total number of CSO in daimain =  310
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)


,geometry,id,author,depth,source,timestamp,elevation,x,y,dem_elev,slope,aspect,lc,tc,Y,M,D,LON,LAT,H
0,POINT (-110.59782 43.67017),D3gyiOmE,Katie O’Connell,10.00,MountainHub,2019-04-25T02:25:30.037Z,2131.137695,532422.643976,4.835319e+06,2128,8.500000,0.0,6.0,8,2019,4,25,-110.597819,43.670172,100.0
1,POINT (-110.59854 43.67067),SEJG7WBM,Katie O’Connell,72.00,MountainHub,2019-04-25T02:24:32.498Z,2127.995361,532364.144681,4.835374e+06,2124,6.775415,0.0,6.0,6,2019,4,25,-110.598541,43.670671,720.0
2,POINT (-110.59358 43.67015),Vn9QgQA7,Leanne,27.94,MountainHub,2019-04-24T15:56:24.335Z,2146.224609,532764.130403,4.835319e+06,2147,7.386643,0.0,2.0,7,2019,4,24,-110.593583,43.670154,279.4
3,POINT (-110.59122 43.67052),xlFXLIST,Leanne,25.40,MountainHub,2019-04-24T15:53:28.932Z,2156.758057,532954.088693,4.835360e+06,2154,11.259718,0.0,1.0,9,2019,4,24,-110.591224,43.670517,254.0
4,POINT (-110.59858 43.66764),4EMEMrPA,Colton Lewer,60.00,MountainHub,2019-04-24T15:53:19.864Z,2188.988770,532362.586657,4.835038e+06,2178,17.787811,0.0,1.0,15,2019,4,24,-110.598580,43.667641,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,POINT (-110.94137 43.78968),RlB3wF6G,Fred Most,56.00,SnowPilot,2018-11-12T16:39:00.000Z,2760.414795,504716.859665,4.848515e+06,2811,29.904013,6.0,1.0,30,2018,11,12,-110.941374,43.789676,560.0
306,POINT (-110.95167 43.47857),6+xKcZFv,Chris McCollister,68.00,SnowPilot,2018-11-09T00:20:00.000Z,2794.276123,503908.588273,4.813962e+06,2805,7.976528,0.0,1.0,6,2018,11,9,-110.951670,43.478566,680.0
307,POINT (-110.85020 43.60553),7MKrMQAz,Josh Pope,95.00,SnowPilot,2018-11-08T17:30:00.000Z,2773.990234,512089.610379,4.828072e+06,2786,65.480194,2.0,6.0,54,2018,11,8,-110.850197,43.605528,950.0
308,POINT (-110.85548 43.60229),BBqIOtV3,Mike Rheam,65.00,SnowPilot,2018-11-08T17:00:00.000Z,2794.865967,511663.718663,4.827712e+06,2861,56.129593,4.0,12.0,45,2018,11,8,-110.855482,43.602289,650.0
